In [1]:
from random import shuffle
train_x = []
train_y = []
test_x = []
test_y = []
for i in range(1,2171):
    with open('data/train/ham/'+str(i)) as f:
        l = f.readlines()
        train_x.append(l[2])
        train_y.append(0)
for i in range(1, 433):
    with open('data/train/spam/'+str(i)) as f:
        l = f.readlines()
        train_x.append(l[2])
        train_y.append(1)
for i in range(1, 243):
    with open('data/test/ham/'+str(i)) as f:
        l = f.readlines()
        test_x.append(l[2])
        test_y.append(0)
for i in range(1, 50):
    with open('data/test/spam/'+str(i)) as f:
        l = f.readlines()
        test_x.append(l[2])
        test_y.append(1)

print(len(train_x), len(train_y), len(test_x), len(test_y))

2602 2602 291 291


In [2]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn import preprocessing
stop = stopwords.words('english')
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer =  WordNetLemmatizer()
for i, j in enumerate(train_x):    
    t = tokenizer.tokenize(j)
    t = [lemmatizer.lemmatize(x.lower(), pos='v') for x in t if x not in stop]
    train_x[i] = ' '.join(t)
for i, j in enumerate(test_x):
    t = tokenizer.tokenize(j)
    t = [lemmatizer.lemmatize(x.lower(), pos='v') for x in t if x not in stop]
    test_x[i] = ' '.join(t)

In [190]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
t = TfidfVectorizer(max_features=1000)
t.fit(train_x)
train_x = np.c_[np.array(t.transform(train_x).toarray())]
test_x = np.c_[np.array(t.transform(test_x).toarray())]
train_y = np.array(train_y)
test_y = np.array(test_y)

# some shuffling
idx = np.arange(train_x.shape[0])
np.random.seed(13)
np.random.shuffle(idx)
train_x = train_x[idx]
train_y = train_y[idx]

print(train_x.shape, test_x.shape)

(2602, 1000) (291, 1000)


In [191]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(class_weight='balanced', n_jobs=-1)
lr.fit(train_x, train_y)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [192]:
from sklearn.metrics import accuracy_score, precision_score
p = lr.predict(test_x)
print(accuracy_score(test_y, p))

0.996563573883


In [193]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
p = gnb.fit(train_x, train_y).predict(test_x)
print(accuracy_score(test_y, p))

0.958762886598


In [194]:
from sklearn import svm
s = svm.LinearSVC()
p = s.fit(train_x, train_y).predict(test_x)
print(accuracy_score(test_y, p))

0.989690721649
